In [4]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.layers import Dense,LeakyReLU,Reshape,Conv2D,Conv2DTranspose,Dense,Flatten,Dropout
from tensorflow.keras import layers
import os
from keras.preprocessing import image

# DC Gan on the cifar 10 dataset

In [5]:
"""hyper parameters"""
latent_dim = 32
height = 32
width = 32
channels = 3

In [7]:
"""G_Net -> the generator network -> a conv net"""
generator_input = keras.Input(shape=(latent_dim,))


S = Dense(128 * 16 * 16)(generator_input)
S = LeakyReLU()(S)
S = Reshape((16, 16, 128))(S)


S = Conv2D(256, 5, padding='same')(S)
S = LeakyReLU()(S)


S = Conv2DTranspose(256, 4, strides=2, padding='same')(S)
S = LeakyReLU()(S)


S = Conv2D(256, 5, padding='same')(S)
S = LeakyReLU()(S)
S = Conv2D(256, 5, padding='same')(S)
S = LeakyReLU()(S)


S = Conv2D(channels, 7, activation='tanh', padding='same')(S)
G_Net = keras.models.Model(generator_input, S)
G_Net.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense_1 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_1 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048

In [8]:
"""D_Net the discriminator for the gans"""
discriminator_input = layers.Input(shape=(height, width, channels))
S = Conv2D(128, 3)(discriminator_input)
S = LeakyReLU()(S)
S = Conv2D(128, 4, strides=2)(S)
S = LeakyReLU()(S)
S = Conv2D(128, 4, strides=2)(S)
S = LeakyReLU()(S)
S = Conv2D(128, 4, strides=2)(S)
S = LeakyReLU()(S)
S = Flatten()(S)


S = Dropout(0.4)(S)


S = Dense(1, activation='sigmoid')(S)  

D_Net = keras.models.Model(discriminator_input, S)
D_Net.summary()

discriminator_optimizer = keras.optimizers.RMSprop(lr=0.0008, clipvalue=1.0, decay=1e-8)
D_Net.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 6, 6, 128)         0   

In [9]:
D_Net.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = D_Net(G_Net(gan_input))
gans = keras.models.Model(gan_input, gan_output)

gan_optimizer = keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gans.compile(optimizer=gan_optimizer, loss='binary_crossentropy')


In [10]:
"""Loading the cifar 10 datasaet"""
(S_train, y_train), (_, _) = keras.datasets.cifar10.load_data()


S_train = S_train[y_train.flatten() == 5]


S_train = S_train.reshape(
    (S_train.shape[0],) + (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = '/home/user/GAN_DC_MNIST/'


In [ ]:
"""Training the network"""
start = 0

for step in range(iterations):
    random_latent_vectors = np.random.normal(size = (batch_size,latent_dim))
    generated_images = G_Net.predict(random_latent_vectors)
    stop = start + batch_size
    real_images = S_train[start:stop]
    combined_images = np.concatenate([generated_images,real_images])
    
    labels = np.concatenate([np.ones((batch_size, 1)),
                             np.zeros((batch_size, 1))])
    
    labels += 0.05 * np.random.random(labels.shape)

                            
    d_loss = D_Net.train_on_batch(combined_images,labels)
    random_latent_vectors = np.random.normal(size = (batch_size,latent_dim))
    misleading_targets = np.zeros((batch_size,1))
    
    a_loss = gans.train_on_batch(random_latent_vectors, misleading_targets)
    
    start += batch_size
    if start > len(S_train) - batch_size:
       start = 0

    
    if step % 10 == 0:
        
        gans.save_weights('gan.h5')

        
        print('discriminator loss at step %s: %s' % (step, d_loss))
        print('adversarial loss at step %s: %s' % (step, a_loss))

        
        img = image.array_to_img(generated_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'generated_Dog' + str(step) + '.png'))

        
        img = image.array_to_img(real_images[0] * 255., scale=False)
        img.save(os.path.join(save_dir, 'real_Dog' + str(step) + '.png'))

discriminator loss at step 0: 0.6986822485923767
adversarial loss at step 0: 0.6768385767936707
discriminator loss at step 10: 0.7533338665962219
adversarial loss at step 10: 0.7962182760238647
discriminator loss at step 20: 0.4859583377838135
adversarial loss at step 20: 1.9407641887664795
discriminator loss at step 30: 0.660897970199585
adversarial loss at step 30: 0.6762562990188599
discriminator loss at step 40: 0.7013099789619446
adversarial loss at step 40: 0.6663370728492737
discriminator loss at step 50: 0.6812707781791687
adversarial loss at step 50: 0.6950147151947021
discriminator loss at step 60: 0.6804402470588684
adversarial loss at step 60: 0.7891403436660767
discriminator loss at step 70: 0.7082245945930481
adversarial loss at step 70: 1.6608175039291382
discriminator loss at step 80: 0.7125769853591919
adversarial loss at step 80: 0.9832416772842407
discriminator loss at step 90: 0.6721616983413696
adversarial loss at step 90: 0.8168312907218933
discriminator loss at s

discriminator loss at step 830: 0.7380880117416382
adversarial loss at step 830: 0.8248098492622375
discriminator loss at step 840: 0.7220908999443054
adversarial loss at step 840: 0.7173399925231934
discriminator loss at step 850: 0.6801667213439941
adversarial loss at step 850: 0.7956535220146179
discriminator loss at step 860: 0.6848591566085815
adversarial loss at step 860: 0.7576422691345215
discriminator loss at step 870: 0.7138684988021851
adversarial loss at step 870: 0.7190700769424438
discriminator loss at step 880: 0.6859742403030396
adversarial loss at step 880: 0.6889955997467041
discriminator loss at step 890: 0.689436137676239
adversarial loss at step 890: 0.7250558733940125
discriminator loss at step 900: 0.6937959790229797
adversarial loss at step 900: 0.7711201906204224
discriminator loss at step 910: 0.6993324160575867
adversarial loss at step 910: 0.7685433626174927
discriminator loss at step 920: 0.6962539553642273
adversarial loss at step 920: 0.7620679140090942
d

discriminator loss at step 1640: 0.6956079602241516
adversarial loss at step 1640: 0.7413477897644043
discriminator loss at step 1650: 0.6911591291427612
adversarial loss at step 1650: 0.7664481997489929
discriminator loss at step 1660: 0.6949762105941772
adversarial loss at step 1660: 0.7447986006736755
discriminator loss at step 1670: 0.7027126550674438
adversarial loss at step 1670: 0.7398961782455444
discriminator loss at step 1680: 0.6882286071777344
adversarial loss at step 1680: 0.7280006408691406
discriminator loss at step 1690: 0.6955105066299438
adversarial loss at step 1690: 0.7617189288139343
discriminator loss at step 1700: 0.6966027021408081
adversarial loss at step 1700: 0.8054065704345703
discriminator loss at step 1710: 0.6768878698348999
adversarial loss at step 1710: 0.8281599879264832
discriminator loss at step 1720: 0.7005088925361633
adversarial loss at step 1720: 0.7682393789291382
discriminator loss at step 1730: 0.7019973397254944
adversarial loss at step 1730:

discriminator loss at step 2450: 0.6920853853225708
adversarial loss at step 2450: 0.6830207705497742
discriminator loss at step 2460: 0.6935537457466125
adversarial loss at step 2460: 0.7172161340713501
discriminator loss at step 2470: 0.6835787892341614
adversarial loss at step 2470: 0.7800552248954773
discriminator loss at step 2480: 0.6997838616371155
adversarial loss at step 2480: 0.9042618870735168
discriminator loss at step 2490: 0.6845744848251343
adversarial loss at step 2490: 0.7355608344078064
discriminator loss at step 2500: 0.6823652982711792
adversarial loss at step 2500: 0.7830003499984741
discriminator loss at step 2510: 0.6712144017219543
adversarial loss at step 2510: 0.7139980792999268
discriminator loss at step 2520: 0.6969900131225586
adversarial loss at step 2520: 0.8317155838012695
discriminator loss at step 2530: 0.7083925604820251
adversarial loss at step 2530: 0.8681570887565613
discriminator loss at step 2540: 0.6789073944091797
adversarial loss at step 2540: